In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import make_scorer
from sklearn.impute import SimpleImputer

In [2]:
X_train_data = pd.read_csv('X_train_feature.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test_feature.csv')

indices_test = np.array(X_test_data)[:,0]
indices_train = np.array(X_train_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(5117, 936)
(5117,)
(3411, 936)


In [3]:
# print((y_train==0).sum())
# print((y_train==1).sum())
# print((y_train==2).sum())
# print((y_train==3).sum())

# train_dataset = np.hstack((X_train, np.reshape(y_train, (5117, 1))))
# print(train_dataset.shape)
    
# class_one = np.zeros((443,937))
# index = 0
# for row in train_dataset:
#     if row[936] == 1:
#         class_one[index] = row
#         index += 1
# class_two = np.zeros((1474,937))
# index = 0
# for row in train_dataset:
#     if row[936] == 2:
#         class_two[index] = row
#         index += 1
# class_three = np.zeros((170,937))
# index = 0
# for row in train_dataset:
#     if row[936] == 3:
#         class_three[index] = row
#         index += 1
        
# for i in range(5):
#     train_dataset = np.vstack((train_dataset, class_one))
# train_dataset = np.vstack((train_dataset, class_two))
# for i in range(10):
#     train_dataset = np.vstack((train_dataset, class_three))
# print(train_dataset.shape)

# X_train = train_dataset[:,:936]
# y_train = train_dataset[:,936]

In [16]:
impute1 = SimpleImputer(strategy = 'median', fill_value = 0)
X_train = impute1.fit_transform(X_train)
impute2 = SimpleImputer(strategy = 'median', fill_value = 0)
X_test = impute2.fit_transform(X_test)

#rescaling data
scaler = StandardScaler() 
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


#using best parameter given by GS

def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test):
    
    #using best parameter given by GS
#     estimator = GradientBoostingClassifier(n_estimators = 250, 
#                                            max_depth = 5,
#                                            learning_rate = 0.1, 
#                                            max_features = 60,
#                                            random_state = 0
#                                           )
    estimator = RandomForestClassifier(random_state=0,
                                       min_samples_leaf=2,
                                       n_estimators=1000,
                                      )
    estimator.fit(X_train, y_train)
    y_val_pred = estimator.predict(X_val)
    score = f1_score(y_val, y_val_pred, average='micro')  
    y_pred = estimator.predict(X_test)

    return score, y_pred

def train_k_fold(X, y, X_test, fold_num=5):
    kf = KFold(n_splits=fold_num, random_state=None, shuffle=False)
    kf.get_n_splits(X)
    test_score = 0.0
    y_pred_list = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test)
        y_pred_list.append(y_pred)
        print('The obtained validation r1 score is : ',score)
        test_score += score
    print("Validation score: %f"%(test_score/fold_num))
    
    y_pred_list = np.array(y_pred_list)
    y_test_predict = []
    for i in range(y_pred_list.shape[1]):
        item = y_pred_list[:,i]
        a = item[item==0].shape
        b = item[item==1].shape
        c = item[item==2].shape
        d = item[item==3].shape
        candidate = [a, b, c, d]
        y_test_predict.append(np.argmax(candidate))
    y_test_predict = np.array(y_test_predict)
    return test_score/fold_num, y_test_predict

score, predictions = train_k_fold(X_train, y_train, X_test)

# estimator = GradientBoostingClassifier(n_estimators = 250, 
#                                            max_depth = 5,
#                                            learning_rate = 0.1, 
#                                            max_features = 60,
#                                            random_state = 0
#                                       )

# estimator.fit(X_train, y_train)
# predictions= estimator.predict(X_test)
sample =  pd.read_csv("sample.csv")
sample["y"] = predictions
sample.to_csv("output.csv", index = False)

KeyboardInterrupt: 